# CODA-database-sandbox

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

#### Date: 11 September 2024

# detect compute platform

In [1]:
import os
import socket

def get_platform():
    hostname = socket.gethostname()
    if "gadi" in hostname:  # Adjust this condition to fit your HPC's hostname or unique identifier
        return 'HPC',hostname
    else:
        return 'Laptop',hostname
    
[platform,hostname] = get_platform()
print('the platform we are working on is '+platform+' with hostname: '+hostname)

the platform we are working on is HPC with hostname: gadi-cpu-bdw-0084.gadi.nci.org.au


### import packages

In [2]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd

In [3]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36139,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:45293,Total threads: 4
Dashboard: /proxy/40377/status,Memory: 35.88 GiB
Nanny: tcp://127.0.0.1:39775,


2024-09-16 17:11:36,503 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:33415'.
2024-09-16 17:11:36,505 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:38833'.
2024-09-16 17:11:36,506 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:45359'.
2024-09-16 17:11:36,508 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:33277'.
2024-09-16 17:11:36,509 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:46701'.
2024-09-16 17:11:36,512 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:45293'.
2024-09-16 17:11:36,514 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:44693'.
2024-09-16 17:11:37,760 - distributed.nanny - WARNING - Restarting worker
2024-09-16 17:11:37,826

In [5]:
if platform == 'HPC':
    data_path = '/g/data/es60/users/thomas_moore/CODA/2005/'
    write_path = '/g/data/es60/users/thomas_moore/CODA/parquet/'
else:
    data_path = '/Users/moo270/data/CARSv2/CODA/'
    write_path = '/Users/moo270/data/CARSv2/CODA/parquet/'

print(f"Using data path: {data_path}")
print(f"Using write path: {write_path}")

Using data path: /g/data/es60/users/thomas_moore/CODA/2005/
Using write path: /g/data/es60/users/thomas_moore/CODA/parquet/


In [6]:
ds = xr.open_dataset(data_path+ "WOD2018_CODA_2005_pfl.nc",chunks="auto")

In [7]:
ds

<xarray.Dataset> Size: 5GB
Dimensions:                 (cast: 64100, z_index: 1005)
Coordinates:
  * cast                    (cast) int64 513kB 1 2 3 4 ... 64098 64099 64100
    time                    (cast) datetime64[ns] 513kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lat                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lon                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
  * z_index                 (z_index) int64 8kB 0 1 2 3 ... 1001 1002 1003 1004
    z                       (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
Data variables: (12/29)
    WOD_id                  (cast) int32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    origflagset             (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    country                 (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    dataset                 (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Access_no               (cast) float64 513kB dask.array<chunksize=(64100,), meta=np.ndarray>
    Recorder                (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    ...                      ...
    Chlorophyll_WODflag     (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Chlorophyll_origflag    (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    z_WODflag               (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_origflag              (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    crs                     int32 4B ...
    CODA_id                 (cast) |S20 1MB dask.array<chunksize=(64100,), meta=np.ndarray>
Attributes:
    Parent ragged array file:  wod_pfl_2005.nc

In [8]:
ds = ds.set_coords(['WOD_id',
                    'origflagset',
                    'country',
                    'dataset',
                    'Access_no',
                    'dbase_orig',
                    'Project',
                    'WOD_cruise_identifier',
                    'Institute',
                    'Ocean_Vehicle',
                    'Temperature_Instrument',
                    'CODA_id'])

In [9]:
ds

<xarray.Dataset> Size: 5GB
Dimensions:                 (cast: 64100, z_index: 1005)
Coordinates: (12/18)
  * cast                    (cast) int64 513kB 1 2 3 4 ... 64098 64099 64100
    WOD_id                  (cast) int32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    time                    (cast) datetime64[ns] 513kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lat                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    lon                     (cast) float32 256kB dask.array<chunksize=(64100,), meta=np.ndarray>
    origflagset             (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    ...                      ...
    Institute               (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Ocean_Vehicle           (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Temperature_Instrument  (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
  * z_index                 (z_index) int64 8kB 0 1 2 3 ... 1001 1002 1003 1004
    z                       (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    CODA_id                 (cast) |S20 1MB dask.array<chunksize=(64100,), meta=np.ndarray>
Data variables: (12/17)
    Recorder                (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Platform                (cast) |S100 6MB dask.array<chunksize=(64100,), meta=np.ndarray>
    Oxygen                  (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    Oxygen_WODflag          (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Oxygen_origflag         (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    Temperature             (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    ...                      ...
    Chlorophyll             (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    Chlorophyll_WODflag     (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    Chlorophyll_origflag    (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    z_WODflag               (cast, z_index) float64 515MB dask.array<chunksize=(32766, 512), meta=np.ndarray>
    z_origflag              (cast, z_index) float32 258MB dask.array<chunksize=(46280, 725), meta=np.ndarray>
    crs                     int32 4B ...
Attributes:
    Parent ragged array file:  wod_pfl_2005.nc

# make truncated version of ds

In [10]:
ds_truncated = ds#.isel(cast=slice(0,10000))

In [11]:
%%time
df = ds_truncated[['Temperature']].to_dataframe().reset_index()
df

CPU times: user 1min 48s, sys: 52.2 s, total: 2min 41s
Wall time: 2min 43s


,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
0,1,0,27.816999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',4.473917,b'WODpfl200501010001'
1,1,1,27.745001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',9.444819,b'WODpfl200501010001'
2,1,2,27.408001,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',13.918531,b'WODpfl200501010001'
3,1,3,27.285999,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',18.889204,b'WODpfl200501010001'
4,1,4,27.233000,10405561,2005-01-01 00:42:11.160091648,-13.641000,69.829002,b'ARGO profiling floats',b'JAPAN',b'profiling float',42682.0,b'US GODAE server (Argo)',b'J-ARGO (JAPAN ARGO)',b'JP031068',b'JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TE...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',24.257399,b'WODpfl200501010001'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64420495,64100,1000,NaN,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',NaN,b'WODpfl200512310207'
64420496,64100,1001,NaN,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',NaN,b'WODpfl200512310207'
64420497,64100,1002,NaN,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',NaN,b'WODpfl200512310207'
64420498,64100,1003,NaN,11898775,2005-12-31 23:00:57.245635584,39.419998,132.580002,b'ARGO profiling floats',b'KOREA; REPUBLIC OF',b'profiling float',42682.0,b'US GODAE server (Argo)',b'',b'KR009692',b'KOREAN OCEAN RESEARCH AND DEVELOPMENT INSTIT...,"b'APEX (Autonomous Profiling Explorer, Webb Re...",b'CTD: TYPE UNKNOWN',NaN,b'WODpfl200512310207'


# convert to dask dataframe

In [12]:
%%time
ddf = dd.from_pandas(df, npartitions=500)

CPU times: user 3min 32s, sys: 30 s, total: 4min 2s
Wall time: 3min 53s


In [13]:
ddf

,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
npartitions=500,,,,,,,,,,,,,,,,,,,
0,int64,int64,float32,int32,datetime64[ns],float32,float32,string,string,string,float64,string,string,string,string,string,string,float32,string
128841,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64291659,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64420499,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# find columns with byte strings and convert ( is this needed when going from ds --> df --> ddf )

In [42]:
ddf.head()

/g/data/es60/users/thomas_moore/miniconda3/envs/tabular_oceans/lib/python3.10/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 40.94 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
0,1,0,27.816999,10405561,2005-01-01 00:42:11.160091648,-13.641,69.829002,ARGO profiling floats,JAPAN,profiling float,42682.0,US GODAE server (Argo),J-ARGO (JAPAN ARGO),JP031068,JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TECH...,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,4.473917,WODpfl200501010001
1,1,1,27.745001,10405561,2005-01-01 00:42:11.160091648,-13.641,69.829002,ARGO profiling floats,JAPAN,profiling float,42682.0,US GODAE server (Argo),J-ARGO (JAPAN ARGO),JP031068,JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TECH...,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,9.444819,WODpfl200501010001
2,1,2,27.408001,10405561,2005-01-01 00:42:11.160091648,-13.641,69.829002,ARGO profiling floats,JAPAN,profiling float,42682.0,US GODAE server (Argo),J-ARGO (JAPAN ARGO),JP031068,JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TECH...,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,13.918531,WODpfl200501010001
3,1,3,27.285999,10405561,2005-01-01 00:42:11.160091648,-13.641,69.829002,ARGO profiling floats,JAPAN,profiling float,42682.0,US GODAE server (Argo),J-ARGO (JAPAN ARGO),JP031068,JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TECH...,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,18.889204,WODpfl200501010001
4,1,4,27.233000,10405561,2005-01-01 00:42:11.160091648,-13.641,69.829002,ARGO profiling floats,JAPAN,profiling float,42682.0,US GODAE server (Argo),J-ARGO (JAPAN ARGO),JP031068,JAPAN AGENCY FOR MARINE-EARTH SCIENCE AND TECH...,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,24.257399,WODpfl200501010001


In [43]:
%%time
ddf.info(memory_usage='deep')

/g/data/es60/users/thomas_moore/miniconda3/envs/tabular_oceans/lib/python3.10/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 20.10 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


<class 'dask_expr.DataFrame'>
Columns: 19 entries, cast to CODA_id
dtypes: datetime64[ns](1), float32(4), float64(1), int32(1), int64(2), string(10)
memory usage: 20.1 GB
CPU times: user 2min 50s, sys: 1min 27s, total: 4min 17s
Wall time: 4min 22s


In [14]:
%%time
ddf.to_parquet(write_path+"2005_pfl_temp.pq",engine='pyarrow', compression='snappy')

/g/data/es60/users/thomas_moore/miniconda3/envs/tabular_oceans/lib/python3.10/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 20.10 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 6min 3s, sys: 2min 40s, total: 8min 44s
Wall time: 8min 37s


In [15]:
%%time
df.to_csv(write_path+"2005_pfl_temp.csv", index=False)

CPU times: user 13min 13s, sys: 3min 4s, total: 16min 18s
Wall time: 13min 19s


# load parquet data & filter

In [16]:
%%time
temp_pfl_ddf = dd.read_parquet(write_path+"2005_pfl_temp.pq")
# Search for rows where 'column_name' matches a condition
result_parquet = temp_pfl_ddf[temp_pfl_ddf ['z'] >= 2000].compute()

CPU times: user 1.68 s, sys: 1.03 s, total: 2.7 s
Wall time: 3.74 s


In [17]:
result_parquet

,cast,z_index,Temperature,WOD_id,time,lat,lon,origflagset,country,dataset,Access_no,dbase_orig,Project,WOD_cruise_identifier,Institute,Ocean_Vehicle,Temperature_Instrument,z,CODA_id
14071,15,1,27.490,10405576,2005-01-01 03:17:59.999827968,-13.809000,81.542000,,UNITED STATES,profiling float,1959.0,GTSP Program,U.S. ARGO PROJECT,US028807,UNIVERSITY OF WASHINGTON; SEATTLE,"APEX (Autonomous Profiling Explorer, Webb Rese...","CTD: Sea-Bird Electronics, MODEL UNKNOWN",3066.000000,WODpfl200501010015
95546,96,71,4.567,10405654,2005-01-01 14:31:51.960982528,36.483002,-14.506000,ARGO profiling floats,SPAIN,profiling float,42682.0,US GODAE server (Argo),,ES001259,,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,2005.941772,WODpfl200501010096
212110,212,55,2.949,10405750,2005-01-02 08:00:57.238769664,-26.886000,-26.415001,ARGO profiling floats,GREAT BRITAIN,profiling float,42682.0,US GODAE server (Argo),ARGO UK,GB011482,,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,2003.782593,WODpfl200501020070
655331,653,71,3.753,10406204,2005-01-05 10:30:00.000000000,44.233002,-9.640000,ARGO profiling floats,SPAIN,profiling float,42682.0,US GODAE server (Argo),,ES001250,,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,2004.039673,WODpfl200501050062
942690,939,0,21.430,10406486,2005-01-07 06:28:00.000113664,-16.613001,-116.598999,,UNITED STATES,profiling float,1970.0,GTSP Program,U.S. ARGO PROJECT,US028886,UNIVERSITY OF WASHINGTON; SEATTLE,"APEX (Autonomous Profiling Explorer, Webb Rese...","CTD: Sea-Bird Electronics, MODEL UNKNOWN",3081.000000,WODpfl200501070050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63447715,63133,55,2.821,10624623,2005-12-27 08:12:11.160735744,-29.524000,-37.417000,ARGO profiling floats,GREAT BRITAIN,profiling float,42682.0,US GODAE server (Argo),ARGO UK,GB011473,,"APEX (Autonomous Profiling Explorer, Webb Rese...",CTD: TYPE UNKNOWN,2004.065308,WODpfl200512270056
63556291,63241,91,3.430,10624770,2005-12-28 00:50:38.039874560,48.310001,-14.918000,ARGO profiling floats,FRANCE,profiling float,42682.0,US GODAE server (Argo),CONGAS (CONTINENTAL GASCOGNE),FR014222,,"PROVOR (free-drifting hydrographic profiler, I...",CTD: TYPE UNKNOWN,2018.901978,WODpfl200512280005
63624662,63309,122,3.516,10624836,2005-12-28 07:04:41.160735744,3.241000,-13.970000,ARGO profiling floats,FRANCE,profiling float,42682.0,US GODAE server (Argo),,FR014496,,"PROVOR (free-drifting hydrographic profiler, I...",CTD: TYPE UNKNOWN,2004.140259,WODpfl200512280073
64275902,63957,122,3.460,10625530,2005-12-31 07:21:32.761230336,-5.114000,-9.137000,ARGO profiling floats,FRANCE,profiling float,42682.0,US GODAE server (Argo),,FR014490,,"PROVOR (free-drifting hydrographic profiler, I...",CTD: TYPE UNKNOWN,2010.000000,WODpfl200512310064


In [18]:
df = pd.read_csv(write_path+"2005_pfl_temp.csv")

Exception ignored in sys.unraisablehook: <built-in function unraisablehook>
Traceback (most recent call last):
  File "/g/data/es60/users/thomas_moore/miniconda3/envs/tabular_oceans/lib/python3.10/site-packages/ipykernel/iostream.py", line 664, in write
  File "/g/data/es60/users/thomas_moore/miniconda3/envs/tabular_oceans/lib/python3.10/site-packages/ipykernel/iostream.py", line 509, in parent_header
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
    return self._parent_header.get()
KeyboardInterrupt: 
Traceback (most recent call last):
Traceback (most recent call last):
  File "/g/data/es60/users/thomas_moore/miniconda3/envs/tabular_oceans/lib/python3.10/site-packages/distributed/compatibility.py", line 236, in asyncio_run
    return loop.ru

# load CSV data and filter

In [20]:
%%time
temp_pfl_ddf = dd.read_csv(write_path+"2005_pfl_temp.csv", blocksize=25e6)  # 25 MB chunks
# Search for rows where 'column_name' matches a condition
result_df = temp_pfl_ddf[temp_pfl_ddf['z'] >= 2000].compute()

CPU times: user 8.5 s, sys: 485 ms, total: 8.99 s
Wall time: 42.2 s


# list of todo's

- create parquet from a df
- consider utility of turning "metadata" vars into coords
- consider removing NaN's at df step
- consider removing NaN's otherwise



In [ ]:
df.dropna(how='all',column='z')

In [ ]:
%%time
df = ds.to_dataframe()#.reset_index()